In [4]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [6]:
import os
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [1]:
from openai import OpenAI

In [2]:
client=OpenAI()

In [ ]:
client.files.create(
    file=open("/content/converted_OETReadingPartB.jsonl", "rb"),
    purpose="fine-tune"
)

In [ ]:
files=client.files.list()

In [ ]:
files=client.files.list()

In [ ]:
for file in files:
  if file.filename=='converted_OETReadingPartB.jsonl':
    training_file_id=file.id
    break


In [ ]:
client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-VacpAbs9HEeVTlYumyQ8zOaO', created_at=1729495351, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-pfE7CVTNIVvNT6fd39Az3vpr', result_files=[], seed=555716723, status='validating_files', trained_tokens=None, training_file='file-6UKE3TPjO7QadNqULdhP8kU5', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
jobs=client.fine_tuning.jobs.list()

In [ ]:
for job in jobs:
  print(job)

FineTuningJob(id='ftjob-VacpAbs9HEeVTlYumyQ8zOaO', created_at=1729495351, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-pfE7CVTNIVvNT6fd39Az3vpr', result_files=[], seed=555716723, status='validating_files', trained_tokens=None, training_file='file-6UKE3TPjO7QadNqULdhP8kU5', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
FineTuningJob(id='ftjob-taT3U88hpFekZTo60gFj4mGp', created_at=1729490560, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::AKftMGdA', finished_at=1729490953, hyperparameters=Hyperparameters(n_epochs=9, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-pfE7CVTNIVvNT6fd39Az3vpr', result_files=['f

In [ ]:
client.fine_tuning.jobs.retrieve("ftjob-taT3U88hpFekZTo60gFj4mGp")

FineTuningJob(id='ftjob-taT3U88hpFekZTo60gFj4mGp', created_at=1729490560, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::AKftMGdA', finished_at=1729490953, hyperparameters=Hyperparameters(n_epochs=9, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-pfE7CVTNIVvNT6fd39Az3vpr', result_files=['file-3YYCLTB5E9WerVbrJExE7swH'], seed=1845559494, status='succeeded', trained_tokens=26856, training_file='file-O3pazkyAFpynROc0IdSaquwr', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [3]:
response=client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal::AKftMGdA",# better model
    messages=[
        {"role": "user", "content": "Create a reading comprehension task suitable for the OET Reading Test, specifically for Part B. The task should involve a notice to healthcare staff about upcoming changes to patient documentation procedures, emphasizing the need for accurate recording and compliance. Formulate one multiple-choice question based on this case.with 3 options only"}
    ])

In [4]:
generated_text=response.choices[0].message.content
print(response.choices[0].message.content)


{"Passage": "NOTICE: Starting from March 1st, all patient records must be updated using the new electronic health record (EHR) system. It is mandatory for all clinical staff to ensure that patient notes are entered into the system promptly after each consultation. Failure to do so may result in incomplete medical records, which can jeopardize patient care and violate hospital policies. Training sessions on the new EHR system are available in the staff lounge every weekday until the end of February.", "Question": "What is the consequence of not updating patient notes promptly in the new EHR system?", "Options": {"a": "Incomplete medical records, which can jeopardize patient care.", "b": "Increased workload for other staff members.", "c": "Potential delays in billing for patient services."}, "Correct Answer": "a", "Estimated time": "4 minutes"}


**EVALUATING THE MODEL**

In [12]:
# Example dataset (replace with your actual data)
references = [
    "{\"Passage\": \"NOTICE: From November 1st, all patient records must be updated using the new electronic health record (EHR) system. It is mandatory for all clinical staff to ensure that patient notes are entered into the system promptly after each consultation. Failure to do so may lead to incomplete medical records, which can compromise patient care and violate hospital policies. Staff members are encouraged to attend the EHR training sessions available next week to avoid potential errors.\", \"Question\": \"What is the consequence of not updating patient notes promptly in the new EHR system?\", \"Options\": [\"a) Delayed billing for patients.\", \"b) Incomplete medical records, which can compromise patient care.\", \"c) Increased workload for other staff members.\", \"d) Violations of national healthcare regulations.\"], \"Correct Answer\": \"b) Incomplete medical records, which can compromise patient care.\", \"Estimated time\": \"4 minutes\"}"

]

generated_texts = [
    generated_text,  # This would come from your fine-tuned model's output
]


In [14]:
!pip install rouge-score evaluate


  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e0abcd78b1012dc3d0e12a7b557c63e4a3e12e9aecfaf879c4c2e752f0e820e6
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [15]:
import evaluate

# Load the ROUGE metric from the Hugging Face `evaluate` library
rouge = evaluate.load('rouge')

# Compute ROUGE scores
results = rouge.compute(predictions=generated_texts, references=references)

# Display the results
print("ROUGE Results:", results)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


ROUGE Results: {'rouge1': 0.8705035971223021, 'rouge2': 0.7608695652173914, 'rougeL': 0.8417266187050358, 'rougeLsum': 0.8417266187050358}


CHECKING THE ERRORS IN JSNOL FILE /CALCULATING TOKENS/CHARGES

In [ ]:
!pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.2 MB/s eta 0:00:00


In [8]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [17]:
import json
# data_path = "/content/converted_OETReadingPartB.jsonl"
data_path="D:/EDrivebackup/brocamp/DataScienceProjects/OETPractioner/oet_practicetest/db/OETReadingPartC_final.jsonl"
# data_path="D:/EDrivebackup/brocamp/DataScienceProjects/OETPractioner/oet_practicetest/db/OETReadingPartC_final_copy_2.jsonl"
# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [ ]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [14]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [15]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

NameError: name 'dataset' is not defined

In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~2594 tokens that will be charged for during training
By default, you'll train for 10 epochs on this dataset
By default, you'll be charged for ~25940 tokens
